# import

In [1]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.utils import init_seed, init_logger


## Deep Seek
# from utils.CustomTrainer import CustomTrainer


## ChatGPT
import torch
import pandas as pd
# from recbole.model.general_recommender.bpr import BPR
from recbole.trainer import Trainer


ImportError: cannot import name 'GeneralDataLoader' from 'recbole.data.dataloader.general_dataloader' (c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\data\dataloader\general_dataloader.py)

In [2]:
K = [1,5,10,20]
# VALID_METRIC = 'Recall@'+str(K)
MODEL = 'BPR'
SEED = 2020
USE_GPU = False
SHUFFLE = False 

In [3]:
model_name = MODEL
save_path = 'processed_datasets/natural_data/goodreads/inter_dedup_coldstart_3stars_4x714k/'
base_filename = 'more_2interQ_df'
specs_str = '1959x98333_1.0'

base_dataset_name = base_filename+'_'+specs_str
model_versions = ['_pt1', '_pt2', '_pt3', '_pt4']

# DeepSeek code

The following code yields the error TypeError: 'NegSampleEvalDataLoader' object is not subscriptable

In [ ]:
# for part in model_versions:
#     print('\n\n'+part)
#     dataset_name=base_dataset_name+part

#     parameter_dict = {
#         'dataset': dataset_name+'.inter',
#         'data_path': save_path,
#         'load_col': {'inter': ['user_id', 'item_id']},
#         'use_gpu':USE_GPU,
#         'topk':K,
#         # 'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'], # default
#         # 'valid_metric':VALID_METRIC,
#         # 'user_inter_num_interval':'[1,inf)',
#         'checkpoint_dir':save_path+dataset_name,
#         'seed':SEED,
#         'shuffle': SHUFFLE,
#         'state':'ERROR',
#         # 'show_progress': SHOW_PROGRESS,
#         # https://recbole.io/docs/user_guide/config/evaluation_settings.html
#         'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
#                     'group_by': 'user',
#                     'order': 'RO', # random ordering
#                     'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
#     }


#     # configurations initialization
#     config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

#     # init random seed
#     init_seed(config['seed'], config['reproducibility'])

#     # logger initialization
#     init_logger(config)
#     logger = getLogger()

#     # write config info into log
#     logger.info(config)

#     # dataset creating and filtering
#     dataset = create_dataset(config)
#     logger.info(dataset)

#     # dataset splitting
#     train_data, valid_data, test_data = data_preparation(config, dataset)

#     # model loading and initialization
#     model = BPR(config, train_data.dataset).to(config['device'])
#     logger.info(model)

#     # trainer loading and initialization
#     trainer = CustomTrainer(config, model)

#     # model training
#     best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
#     print('\n\nTraining best results')
#     print('best_valid_score: ', best_valid_score)
#     print('best_valid_result: ', best_valid_result)

#     # model evaluation
#     test_result = trainer.evaluate(test_data, model)

#     print('\n\nTest results')
#     print(test_result)

# ChatGPT

In [ ]:
for part in model_versions:
    print('\n\n'+part)
    dataset_name=base_dataset_name+part

    parameter_dict = {
        'dataset': dataset_name+'.inter',
        'data_path': save_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        # 'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'], # default
        # 'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':save_path+dataset_name,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }


    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)


    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    test_dataloader = GeneralDataLoader(config, dataset=test_data, batch_size=128)

    # Store recommendations
    recommendations = []

    # Generate predictions
    with torch.no_grad():
        for batch in test_dataloader:
            user_ids = batch['user_id']
            item_ids = batch['item_id']
            scores = model.predict(batch)

            for u, i, s in zip(user_ids.tolist(), item_ids.tolist(), scores.tolist()):
                recommendations.append((u, i, s))

    # Convert to DataFrame and save
    df_recommendations = pd.DataFrame(recommendations, columns=['user_id', 'item_id', 'score'])
    df_recommendations.to_csv(parameter_dict['checkpoint_dir']+'/'+'bpr_recommendations.csv', index=False)

    print("Recommendations saved successfully!")


    # Extract the test set interactions
    test_set = test_data.inter_feat[['user_id', 'item_id', 'rating']]  # Adjust column names as needed

    # Save to CSV
    test_set.to_csv(parameter_dict['checkpoint_dir']+'/'+'test_set.csv', index=False)

    print("Test set saved successfully!")

# just checking a little trust-issue-driven question i have, that is whether if the valid set interferes in any way or other parameter. Go back to noteb 09_2
